In [9]:
import os
import requests
import hashlib
from feedparser import parse

In [16]:
def download_podcast_episodes(feed, dir):
    feed = parse(feed)
    for entry in feed.entries:
        url = entry.enclosures[0].href
        title = entry.title
        filename = os.path.join(dir, title + '.mp3')
        if os.path.exists(filename):
            print('Skipping', title)
            continue
        print('Downloading', title)
        r = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(r.content)
        
        # create csv with file names appened to the end
        with open('podcast.csv', 'a') as f:
            f.write(filename + '\n')

In [15]:
# get list of episodes in directory and add to new json file
for files in os.listdir('Spooked'):
    with open('podcast.txt', 'a') as f:
        f.write(files + '\n')

In [10]:
# Function to fetch the podcast feed
def fetch_podcast_feed(feed_url):
    return parse(feed_url)

# Function to create a hash for episode titles (or you can use episode GUIDs)
def get_episode_hash(title):
    return hashlib.md5(title.encode('utf-8')).hexdigest()

# Function to check which episodes have already been downloaded
def get_downloaded_episodes(save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    return set(os.listdir(save_directory))

# Function to download and save the podcast episode
def download_podcast_episodes(feed, save_directory):
    downloaded_episodes = get_downloaded_episodes(save_directory)
    
    for entry in feed.entries:
        episode_title = entry.title
        episode_hash = get_episode_hash(episode_title)
        episode_file = os.path.join(save_directory, episode_hash + ".mp3")  # Assuming MP3 format
        
        # Skip if episode already exists
        if episode_title + ".mp3" in downloaded_episodes:
            print(f"Episode '{episode_title}' already downloaded. Skipping...")
            continue
        
        # Download the episode
        audio_url = entry.enclosures[0]['url']  # Assuming enclosures contain the audio file URL
        response = requests.get(audio_url)
        
        if response.status_code == 200:
            # Save the episode
            with open(episode_file, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded and saved episode '{episode_title}'.")
        else:
            print(f"Failed to download episode '{episode_title}'. Status code: {response.status_code}")

In [17]:
feed_url = "https://feeds.simplecast.com/TBotaapn"  # Replace with actual podcast RSS feed
save_directory = "Spooked"

# Fetch feed, download episodes, and add metadata
feed = fetch_podcast_feed(feed_url)
download_podcast_episodes(feed, save_directory)

AttributeError: object has no attribute 'encode'